In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import text, sequence
from keras import models, layers
import re
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/0.Projects/Korean Language Models/ratings.txt', sep = '\t', encoding='utf-8')

In [86]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [0]:
df.drop('id', axis=1, inplace=True)

In [0]:
df.drop_duplicates(inplace=True)

In [89]:
display(df.head())
print(df.shape)

,document,label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


(194766, 2)


In [0]:
x_train, x_test, y_train, y_test = train_test_split(df['document'], df['label'], test_size=94766, stratify=df['label'], random_state=1228)

In [0]:
del x_test
del y_test

In [0]:
only_text = x_train
label = y_train

In [0]:
del df

In [94]:
only_text.head()

81941     정말 나도 533명의 자녀가있었으면 하는 생각이나도록 감동깊게 감상한 영화중 TOP...
165506                                 우리의 이영재가 왜 이런 영화를...
130197                                          이름만 드래곤볼...
199017    당췌 뭔 내용인지 이해가안감....아무리 옛날영화라 하지만 무슨 80년대 영화도 아...
29305     1.Good-두 주연의 감정 및 액션연기가 일품/2.Bad-음악은 B급 필 충만하고...
Name: document, dtype: object

In [0]:
only_text = only_text.astype('str')

In [0]:
def remove_puct(x):
  
  pattern = re.compile('[^가-힣a-zA-Z0-9]+')
  return re.sub(pattern, ' ', x)

In [0]:
only_text = only_text.apply(remove_puct)

In [0]:
def remove_multisp(x):
  pattern = re.compile('\s\s+')
  return re.sub(pattern, ' ', x.strip())

In [0]:
only_text = only_text.apply(remove_multisp)

In [0]:
# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = ['*', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

ALPHABET_LIST = ['A', 'B', 'C', 'D', 'E','F','G','H','I','J','K','L','N','M','O','P','Q','R','S','T','U','V','W','X','Y','Z']

NUMBER_LIST = ['0','1','2','3','4','5','6','7','8','9']

def korean_to_be_split(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append(CHOSUNG_LIST[ch1])
            r_lst.append(JUNGSUNG_LIST[ch2])
            r_lst.append(JONGSUNG_LIST[ch3])
        else:
            r_lst.append(w)
            r_lst.append('*')
            r_lst.append('*')
    return r_lst
    

In [101]:
only_text = pd.DataFrame(only_text)

only_text.head()

,document
81941,정말 나도 533명의 자녀가있었으면 하는 생각이나도록 감동깊게 감상한 영화중 TOP...
165506,우리의 이영재가 왜 이런 영화를
130197,이름만 드래곤볼
199017,당췌 뭔 내용인지 이해가안감 아무리 옛날영화라 하지만 무슨 80년대 영화도 아니고 ...
29305,1 Good 두 주연의 감정 및 액션연기가 일품 2 Bad 음악은 B급 필 충만하고...


In [0]:
only_text['ja_mo'] = only_text['document'].apply(korean_to_be_split)

In [103]:
only_text.head()

,document,ja_mo
81941,정말 나도 533명의 자녀가있었으면 하는 생각이나도록 감동깊게 감상한 영화중 TOP...,"[ㅈ, ㅓ, ㅇ, ㅁ, ㅏ, ㄹ, , *, *, ㄴ, ㅏ, *, ㄷ, ㅗ, *, ..."
165506,우리의 이영재가 왜 이런 영화를,"[ㅇ, ㅜ, *, ㄹ, ㅣ, *, ㅇ, ㅢ, *, , *, *, ㅇ, ㅣ, *, ..."
130197,이름만 드래곤볼,"[ㅇ, ㅣ, *, ㄹ, ㅡ, ㅁ, ㅁ, ㅏ, ㄴ, , *, *, ㄷ, ㅡ, *, ..."
199017,당췌 뭔 내용인지 이해가안감 아무리 옛날영화라 하지만 무슨 80년대 영화도 아니고 ...,"[ㄷ, ㅏ, ㅇ, ㅊ, ㅞ, *, , *, *, ㅁ, ㅝ, ㄴ, , *, *, ..."
29305,1 Good 두 주연의 감정 및 액션연기가 일품 2 Bad 음악은 B급 필 충만하고...,"[1, *, *, , *, *, G, *, *, o, *, *, o, *, *, ..."


In [116]:
ja_mo = only_text.loc[:, 'ja_mo']

ja_mo.head()

81941     [ㅈ, ㅓ, ㅇ, ㅁ, ㅏ, ㄹ,  , *, *, ㄴ, ㅏ, *, ㄷ, ㅗ, *, ...
165506    [ㅇ, ㅜ, *, ㄹ, ㅣ, *, ㅇ, ㅢ, *,  , *, *, ㅇ, ㅣ, *, ...
130197    [ㅇ, ㅣ, *, ㄹ, ㅡ, ㅁ, ㅁ, ㅏ, ㄴ,  , *, *, ㄷ, ㅡ, *, ...
199017    [ㄷ, ㅏ, ㅇ, ㅊ, ㅞ, *,  , *, *, ㅁ, ㅝ, ㄴ,  , *, *, ...
29305     [1, *, *,  , *, *, G, *, *, o, *, *, o, *, *, ...
Name: ja_mo, dtype: object

In [0]:
ja_mo_list = ja_mo.to_list() + [CHOSUNG_LIST] + [JUNGSUNG_LIST] + [JONGSUNG_LIST]+ [ALPHABET_LIST]+ [NUMBER_LIST]

In [118]:
len(ja_mo_list)

100005

In [119]:
print(ja_mo_list[:1])

[['ㅈ', 'ㅓ', 'ㅇ', 'ㅁ', 'ㅏ', 'ㄹ', ' ', '*', '*', 'ㄴ', 'ㅏ', '*', 'ㄷ', 'ㅗ', '*', ' ', '*', '*', '5', '*', '*', '3', '*', '*', '3', '*', '*', 'ㅁ', 'ㅕ', 'ㅇ', 'ㅇ', 'ㅢ', '*', ' ', '*', '*', 'ㅈ', 'ㅏ', '*', 'ㄴ', 'ㅕ', '*', 'ㄱ', 'ㅏ', '*', 'ㅇ', 'ㅣ', 'ㅆ', 'ㅇ', 'ㅓ', 'ㅆ', 'ㅇ', 'ㅡ', '*', 'ㅁ', 'ㅕ', 'ㄴ', ' ', '*', '*', 'ㅎ', 'ㅏ', '*', 'ㄴ', 'ㅡ', 'ㄴ', ' ', '*', '*', 'ㅅ', 'ㅐ', 'ㅇ', 'ㄱ', 'ㅏ', 'ㄱ', 'ㅇ', 'ㅣ', '*', 'ㄴ', 'ㅏ', '*', 'ㄷ', 'ㅗ', '*', 'ㄹ', 'ㅗ', 'ㄱ', ' ', '*', '*', 'ㄱ', 'ㅏ', 'ㅁ', 'ㄷ', 'ㅗ', 'ㅇ', 'ㄱ', 'ㅣ', 'ㅍ', 'ㄱ', 'ㅔ', '*', ' ', '*', '*', 'ㄱ', 'ㅏ', 'ㅁ', 'ㅅ', 'ㅏ', 'ㅇ', 'ㅎ', 'ㅏ', 'ㄴ', ' ', '*', '*', 'ㅇ', 'ㅕ', 'ㅇ', 'ㅎ', 'ㅘ', '*', 'ㅈ', 'ㅜ', 'ㅇ', ' ', '*', '*', 'T', '*', '*', 'O', '*', '*', 'P', '*', '*', ' ', '*', '*', '1', '*', '*', ' ', '*', '*', 'ㅇ', 'ㅣ', 'ㄴ', ' ', '*', '*', 'ㅇ', 'ㅕ', 'ㅇ', 'ㅎ', 'ㅘ', '*', 'ㅇ', 'ㅣ', 'ㅂ', 'ㄴ', 'ㅣ', '*', 'ㄷ', 'ㅏ', '*', ' ', '*', '*', 'ㅇ', 'ㅣ', '*', 'ㄹ', 'ㅓ', 'ㅎ', 'ㄱ', 'ㅔ', '*', ' ', '*', '*', 'ㄱ', 'ㅏ', 'ㅁ', 'ㄷ', 'ㅗ', 'ㅇ', 'ㅈ', 'ㅓ', 'ㄱ', 'ㅇ', 'ㅣ', '*', 'ㄱ', 'ㅗ', '*', ' ', '*'

In [0]:
tokenizer = text.Tokenizer(filters='', char_level=True)

In [0]:
tokenizer.fit_on_texts(ja_mo_list)

In [122]:
tokenizer.index_word

{1: '*',
 2: 'ㅇ',
 3: ' ',
 4: 'ㅏ',
 5: 'ㄴ',
 6: 'ㅣ',
 7: 'ㄱ',
 8: 'ㄹ',
 9: 'ㅡ',
 10: 'ㅁ',
 11: 'ㅓ',
 12: 'ㅗ',
 13: 'ㅈ',
 14: 'ㄷ',
 15: 'ㅅ',
 16: 'ㅎ',
 17: 'ㅜ',
 18: 'ㅕ',
 19: 'ㅂ',
 20: 'ㅐ',
 21: 'ㅔ',
 22: 'ㅘ',
 23: 'ㅆ',
 24: 'ㅊ',
 25: 'ㅌ',
 26: 'ㅍ',
 27: 'ㅛ',
 28: 'ㄲ',
 29: 'ㅢ',
 30: 'ㅚ',
 31: 'ㅋ',
 32: 'ㅑ',
 33: 'ㄸ',
 34: 'ㅄ',
 35: 'ㅉ',
 36: 'ㅝ',
 37: 'ㅠ',
 38: 'ㄶ',
 39: 'ㅟ',
 40: '1',
 41: 'ㅃ',
 42: 'ㅙ',
 43: '0',
 44: 'ㅖ',
 45: 'o',
 46: '2',
 47: '3',
 48: 't',
 49: 's',
 50: 'e',
 51: '9',
 52: '4',
 53: '5',
 54: 'a',
 55: 'b',
 56: '8',
 57: 'd',
 58: 'c',
 59: '7',
 60: 'g',
 61: 'i',
 62: 'v',
 63: 'ㅀ',
 64: 'ㄻ',
 65: 'n',
 66: 'r',
 67: 'ㄺ',
 68: 'm',
 69: '6',
 70: 'l',
 71: 'ㅞ',
 72: 'f',
 73: 'ㅒ',
 74: 'h',
 75: 'u',
 76: 'p',
 77: 'y',
 78: 'k',
 79: 'w',
 80: 'x',
 81: 'ㄼ',
 82: 'j',
 83: 'z',
 84: 'ㄵ',
 85: 'ㄳ',
 86: 'q',
 87: 'ㄾ',
 88: 'ㄿ',
 89: 'ㄽ'}

In [0]:
train_data = tokenizer.texts_to_sequences(ja_mo.to_list())

In [124]:
max_length = np.max(np.array([len(i) for i in train_data]))

max_length

420

In [0]:
train_data = sequence.pad_sequences(train_data, maxlen= max_length, padding='post')

In [126]:
train_data.shape

(100000, 420)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_data, label, test_size=0.3, stratify = label, random_state=1228)

In [128]:
print(x_train.shape)
print(y_train.shape)

(70000, 420)
(70000,)


In [0]:
from keras import backend as K
import keras

max_len = len(tokenizer.index_word)

input_tensor = keras.layers.Input(shape = (420, ))

x_emb = keras.layers.Embedding(max_len+1, 300)(input_tensor)
x = keras.layers.SpatialDropout1D(0.4)(x_emb)


x_1 = keras.layers.Conv1D(128, 3, padding='same', activation='relu')(x)
x_1_1 = keras.layers.MaxPool1D()(x_1)
x_1_2 = keras.layers.AvgPool1D()(x_1)
concat_x_1 = keras.layers.Concatenate()([x_1_1, x_1_2])

x_2 = keras.layers.Conv1D(128, 6, padding='same', activation='relu')(x)
x_2_1 = keras.layers.MaxPool1D()(x_2)
x_2_2 = keras.layers.AvgPool1D()(x_2)
concat_x_2 = keras.layers.Concatenate()([x_2_1, x_2_2])


x_3 = keras.layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x_3_1 = keras.layers.MaxPool1D()(x_3)
x_3_2 = keras.layers.AvgPool1D()(x_3)
concat_x_3 = keras.layers.Concatenate()([x_3_1, x_3_2])


x_4 = keras.layers.Conv1D(128, 12, padding='same', activation='relu')(x)
x_4_1 = keras.layers.MaxPool1D()(x_4)
x_4_2 = keras.layers.AvgPool1D()(x_4)
concat_x_4 = keras.layers.Concatenate()([x_4_1, x_4_2])


x_5 = keras.layers.Conv1D(128, 15, padding='same', activation='relu')(x)
x_5_1 = keras.layers.MaxPool1D()(x_5)
x_5_2 = keras.layers.AvgPool1D()(x_5)
concat_x_5 = keras.layers.Concatenate()([x_5_1, x_5_2])


concat = keras.layers.Concatenate()([concat_x_1, concat_x_2, concat_x_3, concat_x_4, concat_x_5])



x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(256, return_sequences=True))(concat)


x_1 = layers.GlobalAvgPool1D()(x)
x_2 = layers.GlobalMaxPool1D()(x)
x = layers.Concatenate()([x_1, x_2])

x_out = keras.layers.Dense(512, activation='relu')(x)
x_out = keras.layers.BatchNormalization()(x_out)
x_out = keras.layers.Dense(256, activation='relu')(x_out)
x_out = keras.layers.BatchNormalization()(x_out)
x_out = keras.layers.Dense(128, activation='relu')(x_out)
output = keras.layers.Dense(1, activation='sigmoid')(x_out)



model = keras.models.Model(input_tensor, output)

opt = keras.optimizers.adam(lr = 0.01, beta_1=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# my_call_rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, verbose=1)

In [0]:
history = model.fit(x_train, y_train, batch_size=512, epochs=100, validation_data =(x_test, y_test))




Train on 70000 samples, validate on 30000 samples
Epoch 1/100





70000/70000 [==============================] - 370s 5ms/step - loss: 0.7154 - acc: 0.5305 - val_loss: 1.5702 - val_acc: 0.4994
Epoch 2/100
70000/70000 [==============================] - 348s 5ms/step - loss: 0.6556 - acc: 0.5932 - val_loss: 0.8216 - val_acc: 0.5014
Epoch 3/100
70000/70000 [==============================] - 349s 5ms/step - loss: 0.6248 - acc: 0.6410 - val_loss: 0.6663 - val_acc: 0.6354
Epoch 4/100
70000/70000 [==============================] - 348s 5ms/step - loss: 0.5951 - acc: 0.6734 - val_loss: 5.9448 - val_acc: 0.4994
Epoch 5/100
70000/70000 [==============================] - 347s 5ms/step - loss: 0.5751 - acc: 0.6910 - val_loss: 6.8203 - val_acc: 0.5006
Epoch 6/100
70000/70000 [==============================] - 342s 5ms/step - loss: 0.5498 - acc: 0.7184 - val_loss: 0.6154 - val_acc: 0.6810
Epoch 7/100
70000/70000 [==============================] - 341s 5ms/step - loss: 0.5372 - acc: 0.7282 - val_